In [ ]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from matplotlib import pyplot as plt
import torch
import sys

# Add current directory to path for imports
sys.path.append('.')

# Import utility functions
from utils import (
    # Data preparation
    prepare_data_for_model,
    prepare_data_for_transformer,  # Backward compatibility
    
    # Model creation
    create_model,
    count_parameters,
    DEFAULT_MODEL_CONFIGS,
    
    # Training functions
    setup_training,
    create_checkpoint_dir,
    save_checkpoint,
    train_single_epoch,
    evaluate_model,
    train_water_level_model,
    train_transformer_flood_detection,  # Backward compatibility
    
    # Checkpoint management
    load_checkpoint,
    list_checkpoints,
    resume_training,
    
    # Prediction and evaluation
    batched_inference,
    create_continuous_predictions,
    calculate_metrics,
    
    # Visualization
    plot_training_history,
    plot_predictions,
    plot_error_analysis
)

# Import model classes directly for convenience
from models.transformer import TransformerWaterLevelPrediction
from models.lstm import LSTMFloodDetection

print("Utilities and models imported successfully!")

# Water Level Prediction Training Notebook

This notebook provides a clean interface for training water level prediction models using either Transformer or LSTM architectures.

## Quick Start

1. **Import utilities and set device** (Cell 1)
2. **Configure model type** (Cell 3) - Set `MODEL_TYPE = 'transformer'` or `MODEL_TYPE = 'lstm'`
3. **Run training** (Cell 4)
4. **Visualize results** (Cells 5-8)

## Features

- **Easy model switching**: Just change `MODEL_TYPE` variable
- **Automated data preparation**: Handles normalization and sequence creation
- **Memory-efficient training**: Batched processing with GPU memory management
- **Comprehensive evaluation**: Multiple visualization and metric options
- **Checkpoint management**: Automatic saving and easy model loading

## Available Models

- **Transformer**: Multi-head attention-based model for sequence prediction
- **LSTM**: Bidirectional LSTM with multiple layers

All utility functions are now in `utils.py` for cleaner code organization.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# ===== MODEL CONFIGURATION =====
# Choose model type: 'transformer' or 'lstm'
MODEL_TYPE = 'transformer'  # Change this to 'lstm' to use LSTM model

# ===== TRAINING HYPERPARAMETERS =====
epochs = 10
learning_rate = 0.001
batch_size = 64
evaluation_frequency = 5  # Evaluate every 5 epochs

# ===== COMMON MODEL PARAMETERS =====
input_size = 1  # Single feature (water level)
seq_len = 3*24*10  # 720 time steps
prediction_horizon = 24*10  # Predicting 240 steps ahead (24 hours)
output_size = prediction_horizon  # Predicting multiple future values

# ===== MODEL-SPECIFIC CONFIGURATIONS =====
# You can override default configurations here
custom_model_configs = {
    'transformer': {
        'hidden_dim': 128,
        'num_heads': 8,
        'dim_feedforward': 512,
        'num_layers_enc': 4,
        'num_layers_dec': 4,
        'dropout': 0.1,
    },
    'lstm': {
        'hidden_dim': 128,
        'num_layers': 3,
        'dropout': 0.1,
        'bidirectional': True
    }
}

# Load data - use one of the parquet files for faster loading
try:
    # Try to load from parquet first (much faster)
    df = pd.read_parquet('data/1836026195.parquet')
    print(f"Loaded data from parquet: {df.shape}")
except:
    print("Parquet file not found. Please run the data loading cells first.")
    df = None

if df is not None:
    print(f"Data range: {df['collect_time'].min()} to {df['collect_time'].max()}")
    print(f"Data shape: {df.shape}")
    
    # Prepare data for multi-step prediction
    X_train_src, X_test_src, y_train_tgt, y_test_tgt, scaler = prepare_data_for_model(
        df, 
        seq_len=seq_len, 
        test_size=0.2, 
        prediction_horizon=prediction_horizon
    )

    print(f"\nTarget shapes after data preparation:")
    print(f"y_train_tgt: {y_train_tgt.shape}")
    print(f"y_test_tgt: {y_test_tgt.shape}")

    # Create model using factory function
    model = create_model(
        MODEL_TYPE,
        input_size=input_size,
        output_size=output_size,
        device=device,
        seq_len=seq_len,
        custom_config=custom_model_configs.get(MODEL_TYPE)
    ).to(device)
    
    # Count parameters
    param_stats = count_parameters(model)
    
    print(f"\nModel Summary ({MODEL_TYPE.upper()}):")
    print(f"Total parameters: {param_stats['total']:,}")
    print(f"Trainable parameters: {param_stats['trainable']:,}")
    print(f"Model size: {param_stats['size_mb']:.2f} MB")
    print(f"\nReady to train {MODEL_TYPE} model! Run the next cell to start training.")
else:
    print("Please load data first by running the earlier cells.")

In [ ]:
# Execute training using the modular training function
train_losses, test_losses = train_water_level_model(
    model, X_train_src, y_train_tgt, X_test_src, y_test_tgt,
    model_type=MODEL_TYPE,  # Uses the MODEL_TYPE defined in configuration
    epochs=epochs,
    lr=learning_rate,
    batch_size=batch_size,
    evaluation_frequency=evaluation_frequency,
    device=device
)
print(f"\nTraining completed for {MODEL_TYPE} model!")

In [ ]:
# Plot training history using utility function
plot_training_history(train_losses, test_losses, evaluation_frequency=evaluation_frequency)

print(f"Training epochs: {len(train_losses)} (all epochs)")
print(f"Test evaluations: {len(test_losses)}")

In [ ]:
# Evaluate model performance
model.eval()

# Use batched inference for memory efficiency
print("Running batched inference...")
train_pred = batched_inference(model, X_train_src, batch_size=batch_size, device=device)
test_pred = batched_inference(model, X_test_src, batch_size=batch_size, device=device)

print(f"Prediction shapes:")
print(f"train_pred: {train_pred.shape}")
print(f"test_pred: {test_pred.shape}")

# For multi-step predictions, we need to flatten for inverse transform
if y_train_tgt.dim() == 3:
    y_train_flat = y_train_tgt.squeeze(-1)
    y_test_flat = y_test_tgt.squeeze(-1)
else:
    y_train_flat = y_train_tgt
    y_test_flat = y_test_tgt

# Reshape for inverse transform
train_pred_reshaped = train_pred.cpu().numpy().reshape(-1, 1)
test_pred_reshaped = test_pred.cpu().numpy().reshape(-1, 1)
y_train_reshaped = y_train_flat.cpu().numpy().reshape(-1, 1)
y_test_reshaped = y_test_flat.cpu().numpy().reshape(-1, 1)

# Inverse transform predictions
train_pred_unscaled = scaler.inverse_transform(train_pred_reshaped).flatten()
test_pred_unscaled = scaler.inverse_transform(test_pred_reshaped).flatten()
y_train_unscaled = scaler.inverse_transform(y_train_reshaped).flatten()
y_test_unscaled = scaler.inverse_transform(y_test_reshaped).flatten()

# Extract first-step predictions for cleaner visualization
first_step_pred = test_pred[:, 0].cpu().numpy()
first_step_true = y_test_flat[:, 0].cpu().numpy()
first_pred_unscaled = scaler.inverse_transform(first_step_pred.reshape(-1, 1)).flatten()
first_true_unscaled = scaler.inverse_transform(first_step_true.reshape(-1, 1)).flatten()

# Plot predictions
plot_predictions(first_pred_unscaled, first_true_unscaled, time_step=6/60, n_points=200)

# Calculate and display metrics
metrics = calculate_metrics(first_pred_unscaled, first_true_unscaled)
print(f"\nFirst-Step Prediction Metrics:")
print(f"MSE:  {metrics['mse']:.6f} ft²")
print(f"RMSE: {metrics['rmse']:.6f} ft")
print(f"MAE:  {metrics['mae']:.6f} ft")
print(f"R²:   {metrics['r2']:.4f}")

In [ ]:
# Display comprehensive training results summary
print("=" * 80)
print(f"TRAINING RESULTS SUMMARY - {MODEL_TYPE.upper()} MODEL")
print("=" * 80)
print(f"Training Setup:")
print(f"  Model Type:          {MODEL_TYPE}")
print(f"  Epochs trained:      {len(train_losses)}")
print(f"  Evaluations run:     {len(test_losses)}")
print(f"  Prediction Horizon:  {prediction_horizon} steps ({prediction_horizon * 6 / 60:.1f} hours)")
print(f"  Model Parameters:    {param_stats['total']:,}")
print(f"  Model Size:          {param_stats['size_mb']:.2f} MB")
print()
print(f"📊 PERFORMANCE METRICS (First-Step Predictions):")
print(f"  Predictions:         {len(first_pred_unscaled):,} values")
print(f"  MSE:                 {metrics['mse']:.6f} ft²")
print(f"  RMSE:                {metrics['rmse']:.6f} ft")
print(f"  MAE:                 {metrics['mae']:.6f} ft")
print(f"  R² Score:            {metrics['r2']:.4f}")
print()
print(f"💡 NOTE: First-step predictions shown for clean visualization.")
print(f"   The model actually predicts {prediction_horizon} steps ahead.")
print("=" * 80)

In [ ]:
# Create continuous predictions and perform error analysis
if 'model' in locals() and 'X_test_src' in locals() and 'y_test_tgt' in locals():
    print("Creating continuous predictions from non-overlapping sequences...")
    
    # Get continuous predictions
    continuous_results = create_continuous_predictions(
        model, X_test_src, y_test_tgt, scaler, 
        seq_len=seq_len, 
        prediction_horizon=prediction_horizon,
        device=device
    )
    
    if continuous_results is not None:
        pred_unscaled, true_unscaled = continuous_results
        
        # Plot error analysis
        error_metrics = plot_error_analysis(
            pred_unscaled, 
            true_unscaled, 
            prediction_horizon=prediction_horizon
        )
        
        print(f"\n✅ Continuous Prediction Analysis Complete!")
        print(f"Total predictions: {len(pred_unscaled):,}")
        print(f"Time coverage: {len(pred_unscaled) * 6/60:.1f} hours")
        print(f"Non-overlapping sequences: {len(pred_unscaled) // prediction_horizon:,}")
else:
    print("Please run the training cells first to create the model and test data.")

## How to Switch Between Models

To switch between transformer and LSTM models:

1. **Change the MODEL_TYPE variable** in the configuration cell:
   - `MODEL_TYPE = 'transformer'` for Transformer model
   - `MODEL_TYPE = 'lstm'` for LSTM model

2. **Adjust model-specific parameters** in the `model_configs` dictionary if needed:
   - Transformer: `num_heads`, `dim_feedforward`, `num_layers_enc`, `num_layers_dec`
   - LSTM: `num_layers`, `bidirectional`

3. **Run the cells** in order:
   - Configuration cell (creates the model)
   - Training cell (trains the model)
   - Evaluation cells (analyze results)

4. **Model checkpoints** are saved in separate directories:
   - `model_checkpoints/transformer/` for transformer models
   - `model_checkpoints/lstm/` for LSTM models

### Example: Quick Model Comparison

```python
# To compare both models:
# 1. Train transformer: Set MODEL_TYPE = 'transformer', run training
# 2. Train LSTM: Set MODEL_TYPE = 'lstm', run training
# 3. Use list_checkpoints() to see all saved models
# 4. Load and compare using load_checkpoint()
```

In [ ]:
# Example: Quick model comparison workflow
# Uncomment and run this cell to compare models

"""
# Step 1: Train Transformer Model
MODEL_TYPE = 'transformer'
transformer_model = create_model(MODEL_TYPE).to(device)
print("Training Transformer model...")
transformer_losses = train_water_level_model(
    transformer_model, X_train_src, y_train_tgt, X_test_src, y_test_tgt,
    model_type=MODEL_TYPE,
    epochs=5,  # Use fewer epochs for quick comparison
    lr=learning_rate,
    batch_size=batch_size,
    evaluation_frequency=1,
    device=device
)

# Step 2: Train LSTM Model
MODEL_TYPE = 'lstm'
lstm_model = create_model(MODEL_TYPE).to(device)
print("\nTraining LSTM model...")
lstm_losses = train_water_level_model(
    lstm_model, X_train_src, y_train_tgt, X_test_src, y_test_tgt,
    model_type=MODEL_TYPE,
    epochs=5,  # Use fewer epochs for quick comparison
    lr=learning_rate,
    batch_size=batch_size,
    evaluation_frequency=1,
    device=device
)

# Step 3: Compare Results
print("\nModel Comparison:")
print(f"Transformer - Final Test Loss: {transformer_losses[1][-1]:.6f}")
print(f"LSTM - Final Test Loss: {lstm_losses[1][-1]:.6f}")

# Step 4: List all checkpoints
print("\nAll saved checkpoints:")
all_checkpoints = list_checkpoints()
"""

print("Uncomment the code above to run a quick comparison between models")

## Checkpoint Management

Use these utilities to manage saved models:

In [ ]:
# List all available checkpoints
print("Available checkpoints:")
all_checkpoints = list_checkpoints()

# List checkpoints for a specific model type
print("\nTransformer checkpoints:")
transformer_checkpoints = list_checkpoints(model_type='transformer')

print("\nLSTM checkpoints:")
lstm_checkpoints = list_checkpoints(model_type='lstm')

In [ ]:
# Example: Load a checkpoint
# Uncomment and modify the path to load a specific checkpoint

"""
# Load best model
best_model_path = f'model_checkpoints/{MODEL_TYPE}/best_{MODEL_TYPE}_model.pth'
loaded_model, checkpoint_info = load_checkpoint(best_model_path, device=device)

# Or resume training from a checkpoint
model, total_train_losses, total_test_losses = resume_training(
    checkpoint_path=best_model_path,
    X_train=X_train_src,
    y_train=y_train_tgt,
    X_test=X_test_src,
    y_test=y_test_tgt,
    additional_epochs=10,
    lr=0.0001,
    batch_size=32,
    evaluation_frequency=5,
    device=device
)
"""

print("Checkpoint management utilities are ready to use!")